In [7]:
import pandas as pd
from sodapy import Socrata

pd.set_option('display.width', 1505)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

# to read from local static copy
df = pd.read_csv("~/Downloads/Food_Inspections.csv")
# to pull most recent version and read from the SODA API
client = Socrata("data.cityofchicago.org", None)
results = client.get("4ijn-s7e5", limit=20000)
results_df = pd.DataFrame.from_records(results)

#fetch a restaurant by name
han_bat = results_df.loc[results_df['dba_name'] == 'HAN BAT RESTAURANT']
counts = han_bat[['results']].value_counts()
#fails = han_bat.loc[df['Results'0]=='Fail']
print (han_bat[['results','inspection_date','violations']])
print(counts)


      results          inspection_date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           violations
15299    Pass  2022-03-18T00:00:00.000  39. CONTAMINATION PREVENTED DURING FOOD PREPARATION, STORAGE & DISPLAY - Comments: INSTRUCTED TO CLEAN, REORGANIZE REAR STORAGE ROOM. FOOD AND NON FOOD ITEMS MUST BE STORED 6" OFF THE FLOOR FOR EASY ACCESS ON CLEANING | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: NOTED CARDBOARDS ON FLOOR IN REAR PREP/COOKING AREA. INSTRUCTED TO REMOVE. | 55. PHYSICAL FACILITIES INSTALLED, MAINTA